In [1]:
%pwd

'/Users/ryandevera/data-science/umn_environments/Deeplifting/deeplifting/notebooks'

In [2]:
%cd ../..

/Users/ryandevera/data-science/umn_environments/Deeplifting


In [3]:
%ls

LJ_data/                        high-dimension-paper-results/
LJ_data.tar                     high-dimension-search-results/
README.md                       images/
__pycache__/                    jobs/
algorithm_compare_results/      low-dimension-search-results/
config.py                       models/
data/                           paper-images/
data-queue/                     requirements.txt
data-queue-2023-09-24/          results/
data-queue-2023-09-27/          search_results/
deeplifting/                    tasks.py
deeplifting.png                 test-low-dimension-results.png


In [4]:
import glob as glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import tqdm

In [5]:
minima = [
    -1.0,
    -3.0,
    -6.0,
    -9.103852,
    -12.712062,
    -16.505384,
    -19.821489,
    -24.113360,
    -28.422532,
    -32.765970,
    -37.967600,
    -44.326801,
    -47.845157,
    -52.322627,
    -56.815742,
    -61.317995,
    -66.530949,
    -72.659782,
    -77.1777043,
]

dimensions = 18
k = int(dimensions / 3)
print(minima[k - 2])

-12.712062


# Algorithm Comparison Results

In [8]:
# Algorithm keep columns
algorithm_columns = [
    'problem_name',
    'algorithm',
    'dimensions',
    'hits',
    'time',
    'f',
]

# Results from comparison algorithms
file_directory = './data-queue-2023-09-24/lennard-jones/*/*/*'
files = glob.glob(file_directory)
algorithm_df = pd.read_parquet(files)

# algorithm_df = algorithm_df[algorithm_columns]
algorithm_df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x13,x14,x15,f,algorithm,time,problem_name,hits,dimensions,global_minimum
0,0.608182,2.053934,0.572523,-0.245730,1.372102,0.308847,-0.446221,2.443406,0.583978,0.051235,...,-0.107082,1.716376,1.365684,-9.103852,IPOPT,11.150206,lennard_jones_15d,1,15,-9.103852
1,-0.970041,0.206551,-2.828491,-1.557466,-0.552164,-2.250619,-0.763833,0.056702,-1.737768,-1.724523,...,-1.720547,0.545115,-2.069038,-9.103852,IPOPT,10.607135,lennard_jones_15d,1,15,-9.103852
2,-1.084518,-1.869319,-0.227636,-0.785158,-1.214249,-1.085494,-1.866911,-1.474443,-0.925195,-1.344313,...,-1.579736,-0.443047,-1.254445,-9.103852,IPOPT,10.542315,lennard_jones_15d,1,15,-9.103852
3,-0.541736,-0.771756,0.396253,-0.329810,0.266487,0.033222,-0.385319,-1.494836,-0.444783,-0.804149,...,0.273191,-0.592990,-0.357088,-9.103852,IPOPT,10.548442,lennard_jones_15d,1,15,-9.103852
4,2.957915,0.646753,-0.502900,1.887179,0.949779,-0.630775,2.353179,0.212286,-1.339671,1.643941,...,2.662426,1.292606,-1.369048,-9.103852,IPOPT,10.591020,lennard_jones_15d,1,15,-9.103852


In [9]:
algorithm_df.loc[algorithm_df['dimensions'] == 30]

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x13,x14,x15,f,algorithm,time,problem_name,hits,dimensions,global_minimum
40,-0.139124,1.551402,0.214397,0.448820,1.914528,1.097734,-0.329861,1.122555,1.210388,0.663755,...,0.164148,0.127103,1.309016,-27.479739,IPOPT,13.091254,lennard_jones_30d,0,30,-28.422532
41,0.762481,-0.308873,0.341048,-0.524785,-0.647378,-1.428274,-0.661215,0.224523,-0.725132,-0.322530,...,0.177871,0.229794,-1.461327,-25.427039,IPOPT,13.042453,lennard_jones_30d,0,30,-28.422532
42,-0.337114,-0.487608,-0.062337,-0.497204,-0.320158,-1.145562,-1.116193,0.579507,-1.316314,-1.295325,...,-0.824225,-1.253686,-1.642898,-27.446829,IPOPT,0.885321,lennard_jones_30d,0,30,-28.422532
43,-0.157098,-1.148279,-0.855025,0.350825,0.427801,-0.099335,-1.244981,-0.898302,-0.870134,-0.731791,...,0.328109,-0.675575,0.031073,-26.623320,IPOPT,0.945660,lennard_jones_30d,0,30,-28.422532
44,1.144451,0.348381,0.409504,1.069626,0.532796,-0.680191,0.971864,-0.502893,-0.306706,0.556595,...,0.993440,1.617119,-0.890429,-27.446829,IPOPT,13.051699,lennard_jones_30d,0,30,-28.422532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,-0.185654,1.165175,-0.512632,-0.433252,0.101806,-0.313647,0.425313,-0.370994,0.264622,-0.548363,...,-1.388525,-0.329311,0.051380,-26.579187,Deeplifting-LBFGS,NaN,lennard_jones_30d,0,30,-28.422532
636,-0.543384,1.286121,-0.126804,0.001776,-0.451615,-0.151041,0.179785,0.568744,0.320365,0.795890,...,-0.944244,-0.869449,0.251514,-26.558011,Deeplifting-LBFGS,NaN,lennard_jones_30d,0,30,-28.422532
637,-0.789652,0.476928,-0.140869,0.005745,0.731920,-0.890134,-0.306901,-0.333189,-0.726212,0.767623,...,-0.521366,-0.480281,0.352830,-28.422532,Deeplifting-LBFGS,NaN,lennard_jones_30d,1,30,-28.422532
638,-0.776414,-0.023473,0.986457,-0.377546,0.337364,0.016171,0.576323,0.362864,-0.552044,0.603194,...,-0.360963,0.738929,-1.017862,-26.611451,Deeplifting-LBFGS,NaN,lennard_jones_30d,0,30,-28.422532


In [10]:
# Let's get the algorithm compare results!
algorithm_results_df = algorithm_df.groupby(
    ['problem_name', 'algorithm', 'dimensions']
).agg({'hits': ['count', 'mean'], 'time': 'mean'})
algorithm_results_df.columns = [
    '-'.join(column) for column in algorithm_results_df.columns
]
algorithm_results_df = algorithm_results_df.reset_index()

# Need to verify that all problems have the same number of trials in the end
algorithm_results_df = algorithm_results_df.drop(columns=['hits-count'])
algorithm_results_df.pivot_table(
    index='dimensions', columns='algorithm', values='hits-mean'
)

algorithm,Basinhopping,Deeplifting-LBFGS,Differential Evolution,Dual Annealing,IPOPT
dimensions,,,,,
6,1.0,0.875,1.0,1.0,0.6
9,1.0,1.000,1.0,1.0,0.9
12,1.0,1.000,1.0,1.0,1.0
15,1.0,1.000,1.0,1.0,1.0
18,1.0,0.175,0.1,0.0,0.0
21,1.0,0.425,0.3,0.2,0.2
24,1.0,0.450,0.5,0.4,0.5
27,1.0,0.250,0.2,0.1,0.0
30,0.7,0.275,0.0,0.1,0.0


In [ ]:
# Can we visualize the problem?

In [ ]:
import torch


def pairwise_distances(positions: torch.Tensor) -> torch.Tensor:
    # Assume positions has shape [B, 3N] where B is the batch size and N is the number of atoms
    # Reshaping to get individual atoms' positions of shape [B, N, 3]
    positions = positions.view(positions.shape[0], -1, 3)
    # Compute the pairwise differences
    # Subtracting [B, 1, N, 3] from [B, N, 1, 3] gives [B, N, N, 3]
    deltas = positions.unsqueeze(2) - positions.unsqueeze(1)
    # Norm the differences gives [B, N, N]
    distances = torch.norm(deltas, dim=-1)
    return distances


def cluster_potential(positions: torch.Tensor) -> torch.Tensor:
    # Compute the pairwise distances of atoms
    distances = pairwise_distances(positions)

    # Compute the pairwise cost (1 / dist)^12 - (1 / dist)^ 6
    pairwise_cost = (1 / distances).pow(12) - (1 / distances).pow(6.0)

    # Get the upper triangle matrix (ignoring the diagonal)
    ut_pairwise_cost = torch.triu(pairwise_cost, diagonal=1)

    # 4 * Summutation of the upper triangle of pairwise costs gives potential
    potential = 4 * ut_pairwise_cost.sum(dim=(1, 2))
    return potential

In [ ]:
import requests
import tarfile

# Url of tar containing known global minima
url = 'http://doye.chem.ox.ac.uk/jon/structures/LJ/LJ.tar'
# Where to save the tar -- modify as desired
target_path = 'LJ_data.tar'

# Download
response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(target_path, 'wb') as f:
        f.write(response.raw.read())

# Open file
file = tarfile.open(target_path)

#  By default save the data to the 'LJ_data' folder in the local directory
data_path = f'./{target_path.replace(".tar", "")}'
file.extractall(data_path)

file.close()

In [ ]:
import matplotlib.pyplot as plt
import pandas

# Lists to track atom counts and potentials
atom_counts = []
global_potentials = []

# Iterate from 3 to 67 atoms -- the number visited in the paper
for n_atoms in range(3, 68):
    # File path is simply the nuimber of atoms
    file_path = f'{data_path}/{n_atoms}'
    # Get the positions as a dataframe
    dataframe = pandas.read_csv(file_path, header=None, delim_whitespace=True)
    # Make a positions tensor -- note that we add an initial dimension as the objective function is vectorised
    positions = torch.Tensor(dataframe.to_numpy()).unsqueeze(0)
    # Get the potential
    potential = cluster_potential(positions)

    # Update lists of atom counts and potentials
    atom_counts.append(n_atoms)
    global_potentials.append(potential.item())

# Simple plot
plt.plot(atom_counts, global_potentials)
plt.xlabel('Number of Atoms $N$')
plt.ylabel('Cluster potential $E$')
plt.show()

# Sanity check on the last one
print(
    f'Potential of N={n_atoms} is computed as {potential.item()} vs. published value -347.252007'
)

In [ ]:
from evotorch import Problem
from evotorch.algorithms import SNES

snes_potentials = []

for n_atoms in range(15, 30):
    print(f'Solving case N={n_atoms} with SNES')

    # Set up the problem in vectorised mode
    problem = Problem(
        'min',
        cluster_potential,
        vectorized=True,
        device='cuda:0' if torch.cuda.is_available() else 'cpu',
        dtype=torch.float64,  # Higher precision could be helpful
        solution_length=3 * n_atoms,
        initial_bounds=(-1e-12, 1e-12),  # Taken directly from [2]
        store_solution_stats=True,  # Make sure the problem tracks the best discovered solution, even on GPU
    )

    searcher = SNES(
        problem,
        stdev_init=0.01,  # Taken directly from [2]
        popsize=10 * n_atoms,  # Significantly higher than [2]
        center_learning_rate=0.5,  # Halving value from [2] slows learning
        stdev_learning_rate=0.5,  # Halving value from [2] slows learning
        scale_learning_rate=True,  # Boolean flag means modifying the above learning rates rescales the default
    )
    searcher.run(
        1000 * problem.solution_length
    )  # 2x value used in [2], adjusted for half learning rates

    # Best solution found
    best_potential = problem.status['best_eval']
    # Center is also a good estimate
    center_potential = cluster_potential(searcher.status['center'].cpu().unsqueeze(0))[
        0
    ].item()
    if center_potential < best_potential:
        best_potential = center_potential

    print(f'Found potential {best_potential}')

    snes_potentials.append(best_potential)

In [ ]:
# Simple plot
plt.plot(
    atom_counts[15 : len(snes_potentials)],
    global_potentials[15 : len(snes_potentials)],
    label='Known Optima',
)
plt.plot(
    atom_counts[15 : len(snes_potentials)],
    snes_potentials[15 : len(snes_potentials)],
    label='SNES-discovered Solutions',
)
plt.legend()
plt.xlabel('Number of Atoms $N$')
plt.ylabel('Atom Cluster Potential $E$')
plt.show()

In [8]:
import numpy as np

x = np.random.randn(9)

x = np.reshape(x, (1, -1))
positions = np.reshape(x, (x.shape[0], -1, 3))

# Compute the pairwise differences
deltas = positions[:, :, np.newaxis] - positions[:, np.newaxis, :]

# Norm the differences to get [B, N, N]
distances = np.linalg.norm(deltas, axis=-1) ** 2

# Get the upper triangle matrix (ignoring the diagonal)
distances = np.triu(distances, k=1)

# Provide a mask to eliminate divisions from zero
mask = distances > 0

# Compute the pairwise cost (1 / dist)^12 - (1 / dist)^ 6
result = 1.0 / distances[mask] ** 6 - 1.0 / distances[mask] ** 3
result = 4 * result.sum()
print(result)

-0.6583140746148592


In [9]:
import torch

x = torch.from_numpy(x)
x = x.reshape(1, -1)
positions = x.view(x.shape[0], -1, 3)

# Compute the pairwise differences
# Subtracting [B, 1, N, 3] from [B, N, 1, 3] gives [B, N, N, 3]
deltas = positions.unsqueeze(2) - positions.unsqueeze(1)

# Norm the differences gives [B, N, N]
distances = torch.norm(deltas, dim=-1) ** 2

# Get the upper triangle matrix (ignoring the diagonal)
distances = torch.triu(distances, diagonal=1)

# Provide a mask to eliminate divisions from zero
mask = distances > 0

# Compute the pairwise cost (1 / dist)^12 - (1 / dist)^ 6
result = 1.0 / distances[mask] ** 6 - 1.0 / distances[mask] ** 3
result = 4 * result.sum()

In [10]:
result

tensor(-0.6583, dtype=torch.float64)